*   **Año:** 2024
*   **Alumno/a:** Sampayo Melanie
*   **Legajo:** [LEGAJO]

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [ ]:
import pydantic

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [ ]:
from pydantic import BaseModel
class Client(BaseModel):
  DNI:int
  nacionalidad:str


Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [ ]:
usuario=Client(DNI=39755010, nacionalidad="Argentina")
print(usuario)

DNI=39755010 nacionalidad='Argentina'


Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [ ]:
usuario=Client(DNI="39755010", nacionalidad="Argentina")
print(usuario)

DNI=39755010 nacionalidad='Argentina'


Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [ ]:
!pip install pycountry
from pydantic import BaseModel, validator, ValidationError
from datetime import date
import pycountry

class Client(BaseModel):
    DNI: int
    nacionalidad: str
    fecha_registro: date

    @validator('DNI')
    def dni_must_be_positive(cls, value):
        if value < 0:
            raise ValueError('DNI must be positive')
        return value

    @validator('nacionalidad')
    def nacionalidad_must_be_valid(cls, value):
        if not any(country.alpha_2 == value for country in pycountry.countries):
            raise ValueError(f'Nacionalidad "{value}" is not valid')
        return value

    @validator('fecha_registro')
    def registration_date_cannot_be_in_the_future(cls, value):
        if value > date.today():
            raise ValueError('Registration date cannot be in the future')
        return value


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.4 MB/s eta 0:00:00


<ipython-input-8-7a0f22ec2d18>:11: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('DNI')
<ipython-input-8-7a0f22ec2d18>:17: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('nacionalidad')
<ipython-input-8-7a0f22ec2d18>:23: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more det

In [ ]:
try:
    client_valid = Client(DNI=12345678, nacionalidad='AR', fecha_registro=date(2023, 5, 20))
    print("Caso válido:", client_valid)
except ValidationError as e:
    print(e)


Caso válido: DNI=12345678 nacionalidad='AR' fecha_registro=datetime.date(2023, 5, 20)


In [ ]:
try:
    client_invalid_dni = Client(DNI=-12345678, nacionalidad='AR', fecha_registro=date(2023, 5, 20))
except ValidationError as e:
    print("Caso inválido:", e)


Caso inválido (DNI negativo): 1 validation error for Client
DNI
  Value error, DNI must be positive [type=value_error, input_value=-12345678, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [ ]:
try:
    client_invalid_nationality = Client(DNI=12345678, nacionalidad='ZZ', fecha_registro=date(2023, 5, 20))
except ValidationError as e:
    print("Caso inválido:", e)


Caso inválido: 1 validation error for Client
nacionalidad
  Value error, Nacionalidad "ZZ" is not valid [type=value_error, input_value='ZZ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [ ]:
import pandera as pa
from pandera import DataFrameSchema, Column, Check
import pycountry
from datetime import date
import pandas as pd

def validate_country(series: pd.Series) -> pd.Series:
    valid_countries = {country.alpha_2 for country in pycountry.countries}
    return series.isin(valid_countries)


schema = DataFrameSchema({
    "DNI": Column(int, Check(lambda x: x > 0, error="DNI must be positive")),
    "nacionalidad": Column(
        str,
        Check(validate_country, error="Invalid country code")
    ),
    "fecha_registro": Column(
        pa.DateTime,
        Check(lambda x: x <= pd.Timestamp(date.today()), error="Registration date cannot be in the future")
    ),
})

data = [
    {"DNI": 12345678, "nacionalidad": "AR", "fecha_registro": "2023-05-20"},  # Válido
    {"DNI": -12345678, "nacionalidad": "AR", "fecha_registro": "2023-05-20"},  # DNI negativo
    {"DNI": 12345678, "nacionalidad": "ZZ", "fecha_registro": "2023-05-20"},  # Nacionalidad inválida
    {"DNI": 12345678, "nacionalidad": "AR", "fecha_registro": "2025-01-01"},  # Fecha futura
]

df = pd.DataFrame(data)

df["fecha_registro"] = pd.to_datetime(df["fecha_registro"], errors="coerce")


try:
    validated_df = schema.validate(df, lazy=True)
    print("Todos los datos son válidos.")
    print(validated_df)
except pa.errors.SchemaErrors as e:
    print("Caso inválido:")
    print(e.failure_cases)


Caso inválido:
  schema_context          column                                      check  \
0         Column             DNI                       DNI must be positive   
1         Column    nacionalidad                       Invalid country code   
2         Column  fecha_registro  Registration date cannot be in the future   

   check_number         failure_case  index  
0             0            -12345678      1  
1             0                   ZZ      2  
2             0  2025-01-01 00:00:00      3  
